In [132]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost as xgb

In [133]:
df_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
df_test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
df_greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")

In [134]:
df_train.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [135]:
df_train.shape

(617, 58)

In [136]:
df_test.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
df_test.shape

(5, 57)

In [138]:
df_greeks.head()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [139]:
df_greeks.shape

(617, 6)

In [140]:
# df_train.hist(bins=50, figsize = [20,20])
# plt.show()

### Fill Nans with median

In [141]:
for column in df_train.columns:
    if df_train[column].isna().sum() != 0 and column != "Class":
        df_train[column].fillna(df_train[column].median(), inplace=True)

In [142]:
import copy

def clean_data(df):
    df_copy = copy.deepcopy(df)
    
    df_copy = df_copy.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df_copy = df_copy.drop_duplicates()
    
    return df_copy

In [143]:
def one_hot_encode(df):
    df_copy = copy.deepcopy(df)
    eq_one_hot = pd.get_dummies(df_copy["EJ"])  
    df_copy = df_copy.drop(columns=['EJ'], axis=1)
    df_copy = df_copy.join(eq_one_hot)
    return df_copy

In [144]:
df_train = one_hot_encode(df_train)
df_test = one_hot_encode(df_test)

In [145]:
df_train = clean_data(df_train)
df_test = clean_data(df_test)

In [146]:
df_train.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FS,GB,GE,GF,GH,GI,GL,Class,A,B
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,0,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,1,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,1
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,1
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,0,1


In [147]:
X = df_train.drop(columns=["Id", "Class"], axis=1)
y = df_train['Class']

In [148]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Select all except one-hot encoded "EJ" feature
quantitative_features = X.select_dtypes(include=['float64'])

scaled_features = scaler.fit_transform(quantitative_features)

# Create DataFrame drom scaler output and add encoded "EJ" to it
X_scaled = pd.DataFrame(scaled_features, columns=quantitative_features.columns)
X_scaled[['A', 'B']] = X[['A', 'B']]

X_scaled.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,A,B
0,0.021082,0.102347,0.00000,0.030632,0.0,0.000000,0.000000,0.180337,0.002958,0.047364,...,0.000998,0.000866,0.054959,0.000000,0.013846,0.176983,0.362261,0.005425,0,1
1,0.010541,0.027589,0.00000,0.053864,0.0,0.078048,0.000000,0.284500,0.000000,0.074042,...,0.000000,0.016014,0.039418,0.000000,0.194527,0.274495,0.164135,1.000000,1,0
2,0.063949,0.085715,0.00000,0.046519,0.0,0.160575,0.000000,0.265013,0.000000,0.067011,...,0.000385,0.036139,0.250424,0.011229,0.095035,0.258994,0.180218,0.008910,0,1
3,0.028110,0.127285,0.01918,0.117854,0.0,0.079462,0.000000,0.215235,0.000000,0.048203,...,0.000000,0.006925,0.109565,0.006882,0.014475,0.425143,0.470820,0.007039,0,1
4,0.049192,0.124246,0.00000,0.017417,0.0,0.086300,0.002841,0.000000,0.069008,0.078555,...,0.038597,0.001731,0.093459,0.051588,0.059198,0.500829,0.185841,0.004345,0,1


In [149]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2)

In [150]:
from sklearn.utils import class_weight

weights = class_weight.compute_sample_weight('balanced', y_train)

In [151]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV

# params = {
#     "n_estimators": np.arange(100, 1000, 100),
#     "max_depth": np.arange(1, 12, 2),
#     "min_samples_split": np.arange(2, 5),
#     "min_samples_leaf": np.arange(1, 4),
#     "min_weight_fraction_leaf": np.arange(0.0, 0.5, 0.1),
#     "min_impurity_decrease": np.arange(0.0, 0.5, 0.1)
# }

# rf = RandomForestClassifier(
#     n_jobs=-1,
#     random_state=42
# )

# rf_cv = RandomizedSearchCV(rf, params, n_iter=30, n_jobs=-1, cv=5)
# rf_cv.fit(X_train, y_train, sample_weight=weights)

In [152]:
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

def cross_validate_model(model, X, y, n_folds=10):
    n_folds = 10
    oof = []
    f_imp = []
    losses = []

    weights = class_weight.compute_sample_weight('balanced', y)

    str_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(str_kfold.split(X, y)):
        if fold % 5 == 0 or fold == 9:
            print(f'{"#"*24} Training FOLD {fold+1} {"#"*24}')

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

        watch_list = [(X_train, y_train), (X_valid, y_valid)]
            
        model = model.fit(X_train, y_train, sample_weight=weights[train_idx])

        val_preds = model.predict_proba(X_valid)[:, 1]

        val_score = log_loss(y_valid, val_preds, sample_weight=weights[val_idx])

        idx_pred_target = np.vstack([val_idx, val_preds, y_valid]).T

        f_imp.append({i: j for i, j in zip(X.columns, model.feature_importances_)})
        print(f'{" "*20} Log-loss: {val_score:.5f} {" "*6}')

        losses.append(val_score)
        oof.append(idx_pred_target)

    print('*'*45)
    print(f'Mean Log-loss: {np.mean([log_loss(item[:, 2], item[:, 1], sample_weight=weights[item[:, 0].astype(int)]) for item in oof]):.5f}')
    
    return oof, losses, f_imp

In [153]:
# best_rf = rf_cv.best_estimator_

# rf_oof, rf_losses, rf_feat_imp = cross_validate_model(best_rf, X_scaled, y, n_folds=10)

In [154]:
# plt.plot(rf_losses)

In [155]:
# import xgboost as xgb

# params = {
#     "n_estimators": np.arange(100, 1000, 50),
#     "max_depth": np.arange(1, 15),
#     "learning_rate": np.exp(np.random.uniform(np.log(1e-4), np.log(1e-1), size=100)),
#     "subsample": np.arange(0.1, 0.7, 0.05),
#     "colsample_bytree": np.arange(0.1, 0.7, 0.05),
#     "gamma": np.exp(np.random.uniform(np.log(1e-2), np.log(1e2), size=100)),
#     "reg_alpha": np.random.exponential(scale=0.01, size=100),
#     "reg_lambda": np.random.exponential(scale=0.01, size=100),
# }

# xgb = xgb.XGBClassifier(
#     n_jobs=-1,
#     random_state=42
# )

# xgb_cv = RandomizedSearchCV(xgb, params, n_iter=1000, n_jobs=-1, cv=5)
# xgb_cv.fit(X_train, y_train, sample_weight=weights)

In [156]:
best_xgb = xgb.XGBClassifier(
    subsample=0.6,
    reg_lambda=0.03587252599188036,
    reg_alpha=0.013271514624327959,
    n_estimators=600,
    max_depth=12,
    learning_rate=0.02002390873186151,
    gamma=0.05102136928261078,
    colsample_bytree=0.25,
    n_jobs=-1,
    random_state=42,
)

In [157]:
# best_xgb = xgb_cv.best_estimator_

xgb_oof, xgb_losses, xgb_feat_imp = cross_validate_model(best_xgb, X_scaled, y, n_folds=10)

######################## Training FOLD 1 ########################
                     Log-loss: 0.35039       
                     Log-loss: 0.20369       
                     Log-loss: 0.45848       
                     Log-loss: 0.24820       
                     Log-loss: 0.20685       
######################## Training FOLD 6 ########################
                     Log-loss: 0.24909       
                     Log-loss: 0.64960       
                     Log-loss: 0.26827       
                     Log-loss: 0.32424       
######################## Training FOLD 10 ########################
                     Log-loss: 0.17263       
*********************************************
Mean Log-loss: 0.31315


In [158]:
df_test['B'] = 0

test_idxs = df_test['Id']
# Select all except one-hot encoded "EJ" feature
quantitative_features = df_test.select_dtypes(include=['float64'])

scaled_features = scaler.transform(quantitative_features)

# Create DataFrame drom scaler output and add encoded "EJ" to it
df_test_scaled = pd.DataFrame(scaled_features, columns=quantitative_features.columns)

In [159]:
df_test_scaled[['A', 'B']] = df_test[['A', 'B']]

df_test_scaled.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,A,B
0,-0.013352,-0.006759,-0.046687,-0.005065,-0.047649,-0.018628,-0.002486,-0.095483,-0.000841,-0.032971,...,-0.0004,-0.002164,-0.031153,-0.050964,-0.000091,-0.131415,-0.004717,-0.000051,1,0
1,-0.013352,-0.006759,-0.046687,-0.005065,-0.047649,-0.018628,-0.002486,-0.095483,-0.000841,-0.032971,...,-0.0004,-0.002164,-0.031153,-0.050964,-0.000091,-0.131415,-0.004717,-0.000051,1,0
2,-0.013352,-0.006759,-0.046687,-0.005065,-0.047649,-0.018628,-0.002486,-0.095483,-0.000841,-0.032971,...,-0.0004,-0.002164,-0.031153,-0.050964,-0.000091,-0.131415,-0.004717,-0.000051,1,0
3,-0.013352,-0.006759,-0.046687,-0.005065,-0.047649,-0.018628,-0.002486,-0.095483,-0.000841,-0.032971,...,-0.0004,-0.002164,-0.031153,-0.050964,-0.000091,-0.131415,-0.004717,-0.000051,1,0
4,-0.013352,-0.006759,-0.046687,-0.005065,-0.047649,-0.018628,-0.002486,-0.095483,-0.000841,-0.032971,...,-0.0004,-0.002164,-0.031153,-0.050964,-0.000091,-0.131415,-0.004717,-0.000051,1,0


In [160]:
probabilities = best_xgb.predict_proba(df_test_scaled)


probabilities_class_0 = probabilities[:,0]
probabilities_class_1 = probabilities[:,1]

# Create a DataFrame for submission
submission_df = pd.DataFrame()

# Load the test data Ids into the DataFrame
submission_df["Id"] = test_idxs

print(probabilities_class_0)
print(probabilities_class_1)


# Assign the probabilities to the respective columns
submission_df['class_0'] = probabilities_class_0
submission_df['class_1'] = probabilities_class_1

# Save the DataFrame to a CSV file for submission
submission_df.to_csv('submission.csv', index=False)

submission_df

[0.8291857 0.8291857 0.8291857 0.8291857 0.8291857]
[0.17081425 0.17081425 0.17081425 0.17081425 0.17081425]


,Id,class_0,class_1
0,00eed32682bb,0.829186,0.170814
1,010ebe33f668,0.829186,0.170814
2,02fa521e1838,0.829186,0.170814
3,040e15f562a2,0.829186,0.170814
4,046e85c7cc7f,0.829186,0.170814
